In [1]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

In [2]:
''' Seed '''
seed_everything(42)

''' Data Load '''
train = pd.read_csv("/home/workspace/DACON/Click_predict/data/train3.csv")
test = pd.read_csv("/home/workspace/DACON/Click_predict/data/test3.csv")
submission = pd.read_csv("/home/workspace/DACON/Click_predict/data/sample_submission.csv")

''' 전처리 '''
train = preprocessing(train) 
test = preprocessing(test, True)

---------------- Start MissingValue ----------------


100%|██████████| 17/17 [00:22<00:00,  1.30s/it]


Memory usage of dataframe is 8729.67 MB
Memory usage after optimization is: 5155.96 MB
Decreased by 40.9%
---------------- Start MissingValue ----------------


100%|██████████| 17/17 [00:03<00:00,  4.94it/s]


Memory usage of dataframe is 1350.43 MB
Memory usage after optimization is: 805.06 MB
Decreased by 40.4%


In [3]:
F17_values = test['F17'].value_counts().index

for idx, value in enumerate(tqdm(F17_values)):
    ''' train '''
    sampling_train = train[train['F17'] == value]
    sampling_train_X = sampling_train.drop(columns = ["Click"]) 
    sampling_train_y = sampling_train['Click']

    ''' test '''
    sampling_test = test[test['F17'] == value]

    param = {'max_depth': 25,
    'num_leaves': 306,
    'subsample': 0.9757365622458185,
    'subsample_freq': 8,
    'n_estimators' : 1000,
    'min_child_samples': 136}

    model = lgb.LGBMClassifier(random_state=42, **param)
    model.fit(sampling_train_X, sampling_train_y, eval_metric='AUC')

    pred = model.predict_proba(sampling_test)
    print("index")
    print(sampling_test.index)
    submission.loc[sampling_test.index, 'Click'] = pred[:, 1]
    print(submission)
    submission.to_csv(f'{value}_submission.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [3:47:09<00:00, 1362.94s/it] 


In [4]:
test['F17'].value_counts()

F17
0.239624    1788184
0.172729     626161
0.147705     558546
0.179443     432853
0.249512     305183
0.199585     271059
0.087219     259650
0.111572     245350
0.091919      51542
0.041229         13
Name: count, dtype: int64

In [10]:
unique_1 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.24951171875_submission.csv')
unique_2 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.111572265625_submission.csv')
unique_3 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.147705078125_submission.csv')
unique_4 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.179443359375_submission.csv')
unique_5 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.0919189453125_submission.csv')
unique_6 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.1727294921875_submission.csv')
unique_7 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.1995849609375_submission.csv')
unique_8 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.2396240234375_submission.csv')
unique_9 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.08721923828125_submission.csv')
unique_10 = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/0.041229248046875_submission.csv')

In [9]:
final_pred = unique_1['Click']
final_pred += unique_2['Click']
# final_pred += unique_3['Click']
# final_pred += unique_4['Click']
# final_pred += unique_5['Click']
# final_pred += unique_6['Click']
# final_pred += unique_7['Click']
# final_pred += unique_8['Click']
# final_pred += unique_9['Click']
# final_pred += unique_10['Click']

,ID,Click
0,TEST_0000000,0.000000
1,TEST_0000001,0.000000
2,TEST_0000002,0.000000
3,TEST_0000003,0.356528
4,TEST_0000004,0.000000
...,...,...
4538536,TEST_4538536,0.000000
4538537,TEST_4538537,0.000000
4538538,TEST_4538538,0.000000
4538539,TEST_4538539,0.357456


In [11]:
final_pred = unique_1['Click']
final_pred += unique_2['Click']
final_pred

0          0.639164
1          0.341768
2          0.339905
3          0.713056
4          0.747928
             ...   
4538536    0.142693
4538537    0.694999
4538538    0.034625
4538539    0.714911
4538540    0.497422
Name: Click, Length: 4538541, dtype: float64

In [19]:
unique_8[unique_8['Click'] == 0]

,ID,Click
0,TEST_0000000,0.0
1,TEST_0000001,0.0
2,TEST_0000002,0.0
4,TEST_0000004,0.0
5,TEST_0000005,0.0
...,...,...
4538534,TEST_4538534,0.0
4538535,TEST_4538535,0.0
4538536,TEST_4538536,0.0
4538537,TEST_4538537,0.0


In [22]:
len(unique_5[unique_5['Click'] == 0])

13

In [20]:
unique_10

,ID,Click
0,TEST_0000000,0.319582
1,TEST_0000001,0.170884
2,TEST_0000002,0.169953
3,TEST_0000003,0.356528
4,TEST_0000004,0.373964
...,...,...
4538536,TEST_4538536,0.142693
4538537,TEST_4538537,0.347499
4538538,TEST_4538538,0.017313
4538539,TEST_4538539,0.357456
